In [1]:
%pip install --upgrade pip
!git clone https://github.com/google-research/bleurt.git
!cd bleurt && pip install .

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Cloning into 'bleurt'...
remote: Enumerating objects: 134, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 134 (delta 0), reused 17 (delta 0), pack-reused 116 (from 1)
Receiving objects: 100% (134/134), 31.28 MiB | 39.10 MiB/s, done.
Resolving deltas: 100% (49/49), done.
Processing /content/bleurt
  Preparing metadata (setup.py) ... done
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456764 sha256=4045834a377ebba305a2ef85dd051940b2a5d6fd5bb928d6da35f4ba692cedb7
  Stored in directory: /tmp/pip-ephem-wheel-cache-6npapidw/wheels/49/ab/73/9318ab38d4cd1c732bcea8335d3f8d7c0316c8d07b9084fa85
Successfully built BLEURT


In [2]:
!python3 -m unittest bleurt.score_test
!python3 -m unittest bleurt.score_not_eager_test
!python3 -m unittest bleurt.finetune_test
!python3 -m unittest bleurt.score_files_test

2025-02-10 08:43:49.454763: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-10 08:43:49.478262: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739177029.499927    4659 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739177029.507663    4659 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-10 08:43:49.532208: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [3]:
%cd ..

/


In [4]:
# Downloads the 12-layer distilled model, which is ~3.5X smaller.
# !wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D12.zip .
# !unzip BLEURT-20-D12.zip
# this duplicates something we do later

In [5]:
%ls

bin@                        dev/     lib32@   NGC-DL-CONTAINER-LICENSE  root/  tmp/
boot/                       etc/     lib64@   opt/                      run/   tools/
content/                    home/    libx32@  proc/                     sbin@  usr/
cuda-keyring_1.1-1_all.deb  kaggle/  media/   python-apt/               srv/   var/
datalab/                    lib@     mnt/     python-apt.tar.xz*        sys/


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%ls

bin@                        dev/     lib32@   NGC-DL-CONTAINER-LICENSE  root/  tmp/
boot/                       etc/     lib64@   opt/                      run/   tools/
content/                    home/    libx32@  proc/                     sbin@  usr/
cuda-keyring_1.1-1_all.deb  kaggle/  media/   python-apt/               srv/   var/
datalab/                    lib@     mnt/     python-apt.tar.xz*        sys/


In [8]:
!wget https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D12.zip .
!unzip BLEURT-20-D12.zip

--2025-02-10 08:44:53--  https://storage.googleapis.com/bleurt-oss-21/BLEURT-20-D12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.210.207, 173.194.212.207, 173.194.215.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.210.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 625304855 (596M) [application/octet-stream]
Saving to: ‘BLEURT-20-D12.zip’

BLEURT-20-D12.zip   100%[===================>] 596.34M   247MB/s    in 2.4s    

2025-02-10 08:44:55 (247 MB/s) - ‘BLEURT-20-D12.zip’ saved [625304855/625304855]

--2025-02-10 08:44:55--  http://./
Resolving . (.)... failed: No address associated with hostname.
wget: unable to resolve host address ‘.’
FINISHED --2025-02-10 08:44:55--
Total wall clock time: 2.6s
Downloaded: 1 files, 596M in 2.4s (247 MB/s)
Archive:  BLEURT-20-D12.zip
   creating: BLEURT-20-D12/
  inflating: BLEURT-20-D12/bert_config.json  
  inflating: BLEURT-20-D12/saved_model.pb  
   creating: 

In [9]:
import os
import subprocess

import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Define the base directories
#put your base directory here
base_dir = '/content/drive/MyDrive/Colab Notebooks/Translate_benchmark/gemini_2.0_flash_english_to_all/'
eval_dir = os.path.join(base_dir,'BLEURT_evals')

# Ensure eval_data directory exists
os.makedirs(eval_dir, exist_ok=True)

# Function to extract language from file name
def extract_language(file_name):
    # Assuming the file format is gpt-4o-mini_English_to_<Language>.<extension>
    return file_name.split('_')[-1].split('.')[0]

# List all files in the base directory
files = os.listdir(base_dir)
print(files)

# Filter candidates and references
candidates = sorted([f for f in files if f.endswith('.candidates')])
references = sorted([f for f in files if f.endswith('.references')])
print(candidates)
print(references)

# Ensure we have matching candidate-reference pairs
for candidate, reference in zip(candidates, references):
    # Extract language from the file name (assuming the naming convention matches)
    language = extract_language(candidate)
    print(candidate)
    print(reference)
    # Define output score file path
    score_file = os.path.join(eval_dir, f'{language}_scores.eval')

    # Define the BLEURT command
    bleurt_command = [
        'python3', '-m', 'bleurt.score_files',
        f'-candidate_file={os.path.join(base_dir, candidate)}',
        f'-reference_file={os.path.join(base_dir, reference)}',
        '-bleurt_batch_size=100',
        '-batch_same_length=True',
        '-bleurt_checkpoint=BLEURT-20-D12',
        f'-scores_file={score_file}'
    ]

    # Run the BLEURT command
    print(f"Running BLEURT for {language}...")
    subprocess.run(bleurt_command)
    print(f"Scores for {language} saved to {score_file}")

print("BLEURT evaluation completed for all language pairs.")


Num GPUs Available:  1
['gemini_2.0_flash_English_to_Hindi.candidates', 'gemini_2.0_flash_English_to_Hindi.references', 'gemini_2.0_flash_English_to_Hindi.txt', 'gemini_2.0_flash_English_to_Russian.txt', 'gemini_2.0_flash_English_to_Russian.references', 'gemini_2.0_flash_English_to_Russian.candidates', 'gemini_2.0_flash_English_to_Arabic.references', 'gemini_2.0_flash_English_to_Arabic.txt', 'gemini_2.0_flash_English_to_Arabic.candidates', 'gemini_2.0_flash_English_to_Spanish.txt', 'gemini_2.0_flash_English_to_Spanish.candidates', 'gemini_2.0_flash_English_to_Spanish.references', 'gemini_2.0_flash_English_to_Japanese.txt', 'gemini_2.0_flash_English_to_Japanese.references', 'gemini_2.0_flash_English_to_Japanese.candidates', 'gemini_2.0_flash_English_to_German.references', 'gemini_2.0_flash_English_to_German.txt', 'gemini_2.0_flash_English_to_German.candidates', 'gemini_2.0_flash_English_to_Mandarin Chinese.candidates', 'gemini_2.0_flash_English_to_Mandarin Chinese.references', 'gemini_2